In [ ]:
#default_exp util



In [ ]:
#export
import numpy as np
from numpy.fft import fft2, fftn, ifft2, ifftn, fftshift, ifftshift, fft, ifft
def focused_probe(E, N, d, alpha_rad, defocus_nm, det_pix=40e-6, C3_um=1000, C5_mm=1, tx=0, ty=0, Nedge=15,
                  aperture=True, plot=False):
    emass = 510.99906;  # electron rest mass in keV
    hc = 12.3984244;  # h*c
    lam = hc / np.sqrt(E * 1e-3 * (2 * emass + E * 1e-3))  # in Angstrom
    #    print 'lambda : %g' % lam
    alpha = alpha_rad
    tilt_x = 0
    tilt_y = 0

    phi = {
        '11': 0,
        '22': 0,
        '33': 0.0,
        '44': 0.0,
        '55': 0.0,
        '66': 0.0,
        '31': 0.0,
        '42': 0.0,
        '53': 0.0,
        '64': 0.0,
        '51': 0.0,
        '62': 0.0
    }

    a_dtype = {'names': ['10', '11', '20', '22',
                         '31', '33',
                         '40', '42', '44',
                         '51', '53', '55',
                         '60', '62', '64', '66'],
               'formats': ['f8'] * 16}
    a0 = np.zeros(1, a_dtype)
    a0['10'] = 0
    a0['11'] = 0
    a0['20'] = defocus_nm  # defocus: -60 nm
    a0['22'] = 0
    a0['31'] = 0
    a0['33'] = 0
    a0['22'] = 0
    a0['40'] = C3_um  # C3/spherical aberration: 1000 um
    a0['42'] = 0
    a0['44'] = 0
    a0['60'] = C5_mm  # C5/Chromatic aberration: 1 mm

    dk = 1.0 / (N * d)
    qmax = np.sin(alpha) / lam
    ktm = np.arcsin(qmax * lam)
    detkmax = np.arcsin(lam / (2 * d))
    d_alpha = detkmax / (N / 2)

    z = det_pix * N * d / lam

    print('alpha         [mrad]     = %g' % (alpha * 1000))
    print('alpha_max det [mrad]     = %g' % (detkmax * 1000))

    print('qmax                     = %g' % qmax)
    print('beam  dmin [Angstrom]    = %g' % (1 / qmax / 2))
    print('dkmax                    = %g' % (dk * N / 2))
    print('detec dmin [Angstrom]    = %g' % (1 / (dk * N / 2) / 2))
    print('z                 [m]    = %g' % z)

    scalekmax = d_alpha * 50
    # print 'scale bar     [mrad]     = %g' % (scalekmax*1000)

    kx, ky = np.meshgrid(dk * (-N / 2. + np.arange(N)) + tilt_x, dk *
                         (-N / 2. + np.arange(N)) + tilt_y)

    k2 = np.sqrt(kx ** 2 + ky ** 2)
    # riplot(k2,'k2')
    ktheta = np.arcsin(k2 * lam)
    kphi = np.arctan2(ky, kx)
    # riplot(ktheta,'ktheta')
    scaled_a = a0.copy().view(np.float64)
    scales = np.array([10, 10,  # a_2x, nm->A
                       10, 10,  # a_2x, nm->A
                       10, 10,  # a_3x, nm->A
                       1E4, 1E4, 1E4,  # a_4x, um->A
                       1E4, 1E4, 1E4,  # a_5x, um->A
                       1E7, 1E7, 1E7, 1E7,  # a_6x, mm->A
                       ], dtype=np.float64)
    scaled_a *= scales
    a = scaled_a.view(a_dtype)

    cos = np.cos
    chi = 2.0 * np.pi / lam * (1.0 * (a['11'] * cos(2 * (kphi - phi['11'])) + a['10']) * ktheta + 1.0 / 2 * (
            a['22'] * cos(2 * (kphi - phi['22'])) + a['20']) * ktheta ** 2 +
                               1.0 / 3 * (a['33'] * cos(3 * (kphi - phi['33'])) + a['31'] * cos(
                1 * (kphi - phi['31']))) * ktheta ** 3 +
                               1.0 / 4 * (a['44'] * cos(4 * (kphi - phi['44'])) + a['42'] * cos(
                2 * (kphi - phi['42'])) + a['40']) * ktheta ** 4 +
                               1.0 / 5 * (a['55'] * cos(5 * (kphi - phi['55'])) + a['53'] * cos(
                3 * (kphi - phi['53'])) + a['51'] * cos(1 * (kphi - phi['51']))) * ktheta ** 5 +
                               1.0 / 6 * (a['66'] * cos(6 * (kphi - phi['66'])) + a['64'] * cos(
                4 * (kphi - phi['64'])) + a['62'] * cos(2 * (kphi - phi['62'])) + a['60']) * ktheta ** 6)
    # riplot(chi,'chi')
    if aperture:
        arr = np.zeros((N, N), dtype=np.complex64)
    else:
        arr = np.ones((N, N), dtype=np.complex64)
    arr[ktheta < ktm] = 1
    # riplot(arr,'arr')
    dEdge = Nedge / (qmax / dk);  # fraction of aperture radius that will be smoothed
    # some fancy indexing: pull out array elements that are within
    #    our smoothing edges
    ind = np.bitwise_and((ktheta / ktm > (1 - dEdge)),
                         (ktheta / ktm < (1 + dEdge)))
    arr[ind] = 0.5 * (1 - np.sin(np.pi / (2 * dEdge) * (ktheta[ind] / ktm - 1)))
    arr *= np.exp(-1j * chi);

    arr = fftshift(arr)

    arr_real = fftshift(ifft2(arr))
    arr_real /= np.linalg.norm(arr_real)
    #    arr_real = nd.zoom(arr_real.real,0.5) + 1j * nd.zoom(arr_real.imag,0.5)

    return np.real(arr_real).astype(np.float32), np.imag(arr_real).astype(np.float32), np.real(arr).astype(
        np.float32), np.imag(arr).astype(np.float32)

In [ ]:
#export 
import os

PARAM_PREFIX = 'pars'

class Param(dict):
    """
    Convenience class: a dictionary that gives access to its keys
    through attributes.
    
    Note: dictionaries stored in this class are also automatically converted
    to Param objects:
    >>> p = Param()
    >>> p.x = {}
    >>> p
    Param({})
    
    While dict(p) returns a dictionary, it is not recursive, so it is better in this case
    to use p.todict(). However, p.todict does not check for infinite recursion. So please
    don't store a dictionary (or a Param) inside itself.
    
    BE: Please note also that the recursive behavior of the update function will create
    new references. This will lead inconsistency if other objects refer to dicts or Params
    in the updated Param instance. 
    """
    _display_items_as_attributes = True
    _PREFIX = PARAM_PREFIX

    def __init__(self, __d__=None, **kwargs):
        """
        A Dictionary that enables access to its keys as attributes.
        Same constructor as dict.
        """
        dict.__init__(self)
        if __d__ is not None: self.update(__d__)
        self.update(kwargs)

    def __getstate__(self):
        return self.__dict__.items()

    def __setstate__(self, items):
        for key, val in items:
            self.__dict__[key] = val

    def __repr__(self):
        return "%s(%s)" % (self.__class__.__name__, dict.__repr__(self))

    # def __str__(self):
    #     from .verbose import report
    #     return report(self,depth=7,noheader=True)

    def __setitem__(self, key, value):
        # BE: original behavior modified as implicit conversion may destroy references
        # Use update(value,Convert=True) instead
        # return super(Param, self).__setitem__(key, Param(value) if type(value) == dict else value)
        return super(Param, self).__setitem__(key, value)

    def __getitem__(self, name):
        # item = super(Param, self).__getitem__(name)
        # return Param(item) if type(item) == dict else item
        return super(Param, self).__getitem__(name)

    def __delitem__(self, name):
        return super(Param, self).__delitem__(name)

    def __delattr__(self, name):
        return super(Param, self).__delitem__(name)

    # __getattr__ = __getitem__
    def __getattr__(self, name):
        try:
            return self.__getitem__(name)
        except KeyError as ke:
            raise AttributeError(ke)

    __setattr__ = __setitem__

    def copy(self, depth=0):
        """
        :returns Param: A (recursive) copy of P with depth `depth` 
        """
        d = Param(self)
        if depth > 0:
            for k, v in d.iteritems():
                if isinstance(v, self.__class__): d[k] = v.copy(depth - 1)
        return d

    def __dir__(self):
        """
        Defined to include the keys when using dir(). Useful for
        tab completion in e.g. ipython.
        If you do not wish the dict key's be displayed as attributes
        (although they are still accessible as such) set the class 
        attribute `_display_items_as_attributes` to False. Default is
        True.
        """
        if self._display_items_as_attributes:
            return self.keys()
            # return [item.__dict__.get('name',str(key)) for key,item in self.iteritems()]
        else:
            return []

    def update(self, __d__=None, in_place_depth=0, Convert=False, **kwargs):
        """
        Update Param - almost same behavior as dict.update, except
        that all dictionaries are converted to Param if `Convert` is set 
        to True, and update may occur in-place recursively for other Param
        instances that self refers to.
        
        Parameters
        ----------
        Convert : bool 
                  If True, convert all dict-like values in self also to Param.
                  *WARNING* 
                  This mey result in misdirected references in your environment
        in_place_depth : int 
                  Counter for recursive in-place updates 
                  If the counter reaches zero, the Param to a key is
                  replaced instead of updated
        """

        def _k_v_update(k, v):
            # If an element is itself a dict, convert it to Param
            if Convert and hasattr(v, 'keys'):
                # print 'converting'
                v = Param(v)
            # new key 
            if not k in self:
                self[k] = v
            # If this key already exists and is already dict-like, update it
            elif in_place_depth > 0 and hasattr(v, 'keys') and isinstance(self[k], self.__class__):
                self[k].update(v, in_place_depth - 1)
                """
                if isinstance(self[k],self.__class__):
                    # Param gets recursive in_place updates
                    self[k].update(v, in_place_depth - 1)
                else:
                    # dicts are only updated in-place once
                    self[k].update(v)
                """
            # Otherwise just replace it
            else:
                self[k] = v

        if __d__ is not None:
            if hasattr(__d__, 'keys'):
                # Iterate through dict-like argument
                for k, v in __d__.items():
                    _k_v_update(k, v)

            else:
                # here we assume a (key,value) list.
                for (k, v) in __d__:
                    _k_v_update(k, v)

        for k, v in kwargs.items():
            _k_v_update(k, v)

        return None

    def _to_dict(self, Recursive=False):
        """
        Convert to dictionary (recursively if needed).
        """
        if not Recursive:
            return dict(self)
        else:
            d = dict(self)
            for k, v in d.items():
                if isinstance(v, self.__class__): d[k] = v._to_dict(Recursive)
        return d

    @classmethod
    def _from_dict(cls, dct):
        """
        Make Param from dict. This is similar to the __init__ call
        """
        # p=Param()
        # p.update(dct.copy())
        return Param(dct.copy())

In [ ]:
#export
import numpy as np
re = np.s_[..., 0]
im = np.s_[..., 1]
import torch as th
import torch.nn as nn

def fftshift_checkerboard(w, h):
    re = np.r_[w * [-1, 1]]  # even-numbered rows
    ro = np.r_[w * [1, -1]]  # odd-numbered rows
    return np.row_stack(h * (re, ro))

class ZernikeProbeSingle(nn.Module):
    def __init__(self, q: th.Tensor, lam, fft_shifted=True):
        """
        Creates an aberration surface from aberration coefficients. The output is backpropable

        :param q: 2 x M1 x M2 tensor of x coefficients of reciprocal space
        :param lam: wavelength in Angstrom
        :param C: aberration coefficients
        :return: (maximum size of all aberration tensors) x MY x MX
        """

        super(ZernikeProbeSingle, self).__init__()
        self.q = q
        self.lam = lam
        self.fft_shifted = fft_shifted

        if self.fft_shifted:
            cb = fftshift_checkerboard(self.q.shape[1] // 2, self.q.shape[2] // 2)
            self.cb = th.from_numpy(cb).float().to(q.device)

    def forward(self, C, A):
        chi = cartesian_aberrations_single(self.q[1], self.q[0], self.lam, C)
        Psi = cmul(complex_expi(-chi), make_real(A))

        if self.fft_shifted:
            Psi = complex_mul_real(Psi, self.cb)

        return Psi
    
def aperture(q: th.Tensor, lam, alpha_max, edge=2):
    ktheta = th.asin(q.norm(dim=0) * lam)
    qmax = alpha_max / lam
    dk = q[0][1][0]

    arr = th.zeros_like(q[1])
    arr[ktheta < alpha_max] = 1
    if edge > 0:
        dEdge = edge / (qmax / dk)  # fraction of aperture radius that will be smoothed
        # some fancy indexing: pull out array elements that are within
        #    our smoothing edges
        ind = (ktheta / alpha_max > (1 - dEdge)) * (ktheta / alpha_max < (1 + dEdge))
        arr[ind] = 0.5 * (1 - th.sin(np.pi / (2 * dEdge) * (ktheta[ind] / alpha_max - 1)))
    return arr

def cartesian_aberrations(qx, qy, lam, C):
    """
    Zernike polynomials in the cartesian coordinate system
    :param qx:
    :param qy:
    :param lam: wavelength in Angstrom
    :param C:   12 x D
    :return:
    """

    u = qx * lam
    v = qy * lam
    u2 = u ** 2
    u3 = u ** 3
    u4 = u ** 4
    # u5 = u ** 5

    v2 = v ** 2
    v3 = v ** 3
    v4 = v ** 4
    # v5 = v ** 5

    aberr = Param()
    aberr.C1 = C[0].unsqueeze(1).unsqueeze(1)
    aberr.C12a = C[1].unsqueeze(1).unsqueeze(1)
    aberr.C12b = C[2].unsqueeze(1).unsqueeze(1)
    aberr.C21a = C[3].unsqueeze(1).unsqueeze(1)
    aberr.C21b = C[4].unsqueeze(1).unsqueeze(1)
    aberr.C23a = C[5].unsqueeze(1).unsqueeze(1)
    aberr.C23b = C[6].unsqueeze(1).unsqueeze(1)
    aberr.C3 = C[7].unsqueeze(1).unsqueeze(1)
    aberr.C32a = C[8].unsqueeze(1).unsqueeze(1)
    aberr.C32b = C[9].unsqueeze(1).unsqueeze(1)
    aberr.C34a = C[10].unsqueeze(1).unsqueeze(1)
    aberr.C34b = C[11].unsqueeze(1).unsqueeze(1)

    chi = 0

    # r-2 = x-2 +y-2.
    chi += 1 / 2 * aberr.C1 * (u2 + v2) # r^2
    #r-2 cos(2*phi) = x"2 -y-2.
    # r-2 sin(2*phi) = 2*x*y.
    chi += 1 / 2 * (aberr.C12a * (u2 - v2) + 2 * aberr.C12b * u * v) # r^2 cos(2 phi) + r^2 sin(2 phi)
    # r-3 cos(3*phi) = x-3 -3*x*y'2. r"3 sin(3*phi) = 3*y*x-2 -y-3.
    chi += 1 / 3 * (aberr.C23a * (u3 - 3 * u * v2) + aberr.C23b * (3 * u2 * v - v3))# r^3 cos(3phi) + r^3 sin(3 phi)
    # r-3 cos(phi) = x-3 +x*y-2.
    # r-3 sin(phi) = y*x-2 +y-3.
    chi += 1 / 3 * (aberr.C21a * (u3 + u * v2) + aberr.C21b * (v3 + u2 * v))# r^3 cos(phi) + r^3 sin(phi)
    # r-4 = x-4 +2*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C3 * (u4 + v4 + 2 * u2 * v2)# r^4
    # r-4 cos(4*phi) = x-4 -6*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C34a * (u4 - 6 * u2 * v2 + v4)# r^4 cos(4 phi)
    # r-4 sin(4*phi) = 4*x-3*y -4*x*y-3.
    chi += 1 / 4 * aberr.C34b * (4 * u3 * v - 4 * u * v3) # r^4 sin(4 phi)
    # r-4 cos(2*phi) = x-4 -y-4.
    chi += 1 / 4 * aberr.C32a * (u4 - v4)
    # r-4 sin(2*phi) = 2*x-3*y +2*x*y-3.
    chi += 1 / 4 * aberr.C32b * (2 * u3 * v + 2 * u * v3)
    # r-5 cos(phi) = x-5 +2*x-3*y-2 +x*y-4.
    # r-5 sin(phi) = y*x"4 +2*x-2*y-3 +y-5.
    # r-5 cos(3*phi) = x-5 -2*x-3*y-2 -3*x*y-4.
    # r-5 sin(3*phi) = 3*y*x-4 +2*x-2*y-3 -y-5.
    # r-5 cos(5*phi) = x-5 -10*x-3*y-2 +5*x*y-4.
    # r-5 sin(5*phi) = 5*y*x-4 -10*x-2*y-3 +y-5.

    chi *= 2 * np.pi / lam

    return chi

In [ ]:
#export
def cartesian_aberrations_single(qx, qy, lam, C):
    """
    Zernike polynomials in the cartesian coordinate system
    :param qx:
    :param qy:
    :param lam: wavelength in Angstrom
    :param C:   (12 ,)
    :return:
    """

    u = qx * lam
    v = qy * lam
    u2 = u ** 2
    u3 = u ** 3
    u4 = u ** 4
    # u5 = u ** 5

    v2 = v ** 2
    v3 = v ** 3
    v4 = v ** 4
    # v5 = v ** 5

    aberr = Param()
    aberr.C1 = C[0]
    aberr.C12a = C[1]
    aberr.C12b = C[2]
    aberr.C21a = C[3]
    aberr.C21b = C[4]
    aberr.C23a = C[5]
    aberr.C23b = C[6]
    aberr.C3 = C[7]
    aberr.C32a = C[8]
    aberr.C32b = C[9]
    aberr.C34a = C[10]
    aberr.C34b = C[11]

    chi = 0

    # r-2 = x-2 +y-2.
    chi += 1 / 2 * aberr.C1 * (u2 + v2) # r^2
    #r-2 cos(2*phi) = x"2 -y-2.
    # r-2 sin(2*phi) = 2*x*y.
    chi += 1 / 2 * (aberr.C12a * (u2 - v2) + 2 * aberr.C12b * u * v) # r^2 cos(2 phi) + r^2 sin(2 phi)
    # r-3 cos(3*phi) = x-3 -3*x*y'2. r"3 sin(3*phi) = 3*y*x-2 -y-3.
    chi += 1 / 3 * (aberr.C23a * (u3 - 3 * u * v2) + aberr.C23b * (3 * u2 * v - v3))# r^3 cos(3phi) + r^3 sin(3 phi)
    # r-3 cos(phi) = x-3 +x*y-2.
    # r-3 sin(phi) = y*x-2 +y-3.
    chi += 1 / 3 * (aberr.C21a * (u3 + u * v2) + aberr.C21b * (v3 + u2 * v))# r^3 cos(phi) + r^3 sin(phi)
    # r-4 = x-4 +2*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C3 * (u4 + v4 + 2 * u2 * v2)# r^4
    # r-4 cos(4*phi) = x-4 -6*x-2*y-2 +y-4.
    chi += 1 / 4 * aberr.C34a * (u4 - 6 * u2 * v2 + v4)# r^4 cos(4 phi)
    # r-4 sin(4*phi) = 4*x-3*y -4*x*y-3.
    chi += 1 / 4 * aberr.C34b * (4 * u3 * v - 4 * u * v3) # r^4 sin(4 phi)
    # r-4 cos(2*phi) = x-4 -y-4.
    chi += 1 / 4 * aberr.C32a * (u4 - v4)
    # r-4 sin(2*phi) = 2*x-3*y +2*x*y-3.
    chi += 1 / 4 * aberr.C32b * (2 * u3 * v + 2 * u * v3)
    # r-5 cos(phi) = x-5 +2*x-3*y-2 +x*y-4.
    # r-5 sin(phi) = y*x"4 +2*x-2*y-3 +y-5.
    # r-5 cos(3*phi) = x-5 -2*x-3*y-2 -3*x*y-4.
    # r-5 sin(3*phi) = 3*y*x-4 +2*x-2*y-3 -y-5.
    # r-5 cos(5*phi) = x-5 -10*x-3*y-2 +5*x*y-4.
    # r-5 sin(5*phi) = 5*y*x-4 -10*x-2*y-3 +y-5.

    chi *= 2 * np.pi / lam

    return chi

In [ ]:
#export
from numba import cuda
import math as m
import cmath as cm
def single_sideband_reconstruction(G, Qx_all, Qy_all, Kx_all, Ky_all, aberrations, theta_rot, alpha_rad,
                                   Ψ_Qp, Ψ_Qp_left_sb, Ψ_Qp_right_sb, eps, lam):
    threadsperblock = 2 ** 8
    blockspergrid = m.ceil(np.prod(G.shape) / threadsperblock)
    strides = cp.array((np.array(G.strides) / (G.nbytes / G.size)).astype(np.int))
    scale = 1
    single_sideband_kernel[blockspergrid, threadsperblock](G, strides, Qx_all, Qy_all, Kx_all, Ky_all, aberrations,
                                                           theta_rot, alpha_rad, Ψ_Qp, Ψ_Qp_left_sb,
                                                           Ψ_Qp_right_sb, eps, lam, scale)
    cp.cuda.Device(Ψ_Qp.device).synchronize()


@cuda.jit
def single_sideband_kernel(G, strides, Qx_all, Qy_all, Kx_all, Ky_all, aberrations, theta_rot, alpha,
                           Ψ_Qp, Ψ_Qp_left_sb, Ψ_Qp_right_sb, eps, lam, scale):
    def aperture2(qx, qy, lam, alpha_max, scale):
        qx2 = qx ** 2
        qy2 = qy ** 2
        q = m.sqrt(qx2 + qy2)
        ktheta = m.asin(q * lam)
        return (ktheta < alpha_max) * scale

    def chi3(qy, qx, lam, C):
        """
        Zernike polynomials in the cartesian coordinate system
        :param qx:
        :param qy:
        :param lam: wavelength in Angstrom
        :param C:   (12 ,)
        :return:
        """

        u = qx * lam
        v = qy * lam
        u2 = u ** 2
        u3 = u ** 3
        u4 = u ** 4
        # u5 = u ** 5

        v2 = v ** 2
        v3 = v ** 3
        v4 = v ** 4
        # v5 = v ** 5

        # aberr = Param()
        # aberr.C1 = C[0]
        # aberr.C12a = C[1]
        # aberr.C12b = C[2]
        # aberr.C21a = C[3]
        # aberr.C21b = C[4]
        # aberr.C23a = C[5]
        # aberr.C23b = C[6]
        # aberr.C3 = C[7]
        # aberr.C32a = C[8]
        # aberr.C32b = C[9]
        # aberr.C34a = C[10]
        # aberr.C34b = C[11]

        chi = 0

        # r-2 = x-2 +y-2.
        chi += 1 / 2 * C[0] * (u2 + v2)  # r^2
        # r-2 cos(2*phi) = x"2 -y-2.
        # r-2 sin(2*phi) = 2*x*y.
        chi += 1 / 2 * (C[1] * (u2 - v2) + 2 * C[2] * u * v)  # r^2 cos(2 phi) + r^2 sin(2 phi)
        # r-3 cos(3*phi) = x-3 -3*x*y'2. r"3 sin(3*phi) = 3*y*x-2 -y-3.
        chi += 1 / 3 * (C[5] * (u3 - 3 * u * v2) + C[6] * (3 * u2 * v - v3))  # r^3 cos(3phi) + r^3 sin(3 phi)
        # r-3 cos(phi) = x-3 +x*y-2.
        # r-3 sin(phi) = y*x-2 +y-3.
        chi += 1 / 3 * (C[3] * (u3 + u * v2) + C[4] * (v3 + u2 * v))  # r^3 cos(phi) + r^3 sin(phi)
        # r-4 = x-4 +2*x-2*y-2 +y-4.
        chi += 1 / 4 * C[7] * (u4 + v4 + 2 * u2 * v2)  # r^4
        # r-4 cos(4*phi) = x-4 -6*x-2*y-2 +y-4.
        chi += 1 / 4 * C[10] * (u4 - 6 * u2 * v2 + v4)  # r^4 cos(4 phi)
        # r-4 sin(4*phi) = 4*x-3*y -4*x*y-3.
        chi += 1 / 4 * C[11] * (4 * u3 * v - 4 * u * v3)  # r^4 sin(4 phi)
        # r-4 cos(2*phi) = x-4 -y-4.
        chi += 1 / 4 * C[8] * (u4 - v4)
        # r-4 sin(2*phi) = 2*x-3*y +2*x*y-3.
        chi += 1 / 4 * C[9] * (2 * u3 * v + 2 * u * v3)
        # r-5 cos(phi) = x-5 +2*x-3*y-2 +x*y-4.
        # r-5 sin(phi) = y*x"4 +2*x-2*y-3 +y-5.
        # r-5 cos(3*phi) = x-5 -2*x-3*y-2 -3*x*y-4.
        # r-5 sin(3*phi) = 3*y*x-4 +2*x-2*y-3 -y-5.
        # r-5 cos(5*phi) = x-5 -10*x-3*y-2 +5*x*y-4.
        # r-5 sin(5*phi) = 5*y*x-4 -10*x-2*y-3 +y-5.

        chi *= 2 * np.pi / lam

        return chi

    gs = G.shape
    N = gs[0] * gs[1] * gs[2] * gs[3]
    n = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    iqy = n // strides[0]
    iqx = (n - iqy * strides[0]) // strides[1]
    iky = (n - (iqy * strides[0] + iqx * strides[1])) // strides[2]
    ikx = (n - (iqy * strides[0] + iqx * strides[1] + iky * strides[2])) // strides[3]

    if n < N:

        Qx = Qx_all[iqx]
        Qy = Qy_all[iqy]
        Kx = Kx_all[ikx]
        Ky = Ky_all[iky]

        Qx_rot = Qx * m.cos(theta_rot) - Qy * m.sin(theta_rot)
        Qy_rot = Qx * m.sin(theta_rot) + Qy * m.cos(theta_rot)

        Qx = Qx_rot
        Qy = Qy_rot

        A = aperture2(Ky, Kx, lam, alpha, scale) * cm.exp(-1j * chi3(Ky, Kx, lam, aberrations))
        chi_KplusQ = chi3(Ky + Qy, Kx + Qx, lam, aberrations)
        A_KplusQ = aperture2(Ky + Qy, Kx + Qx, lam, alpha, scale) * cm.exp(-1j * chi_KplusQ)
        chi_KminusQ = chi3(Ky - Qy, Kx - Qx, lam, aberrations)
        A_KminusQ = aperture2(Ky - Qy, Kx - Qx, lam, alpha, scale) * cm.exp(-1j * chi_KminusQ)

        Γ = A.conjugate() * A_KminusQ - A * A_KplusQ.conjugate()

        Kplus = sqrt((Kx + Qx) ** 2 + (Ky + Qy) ** 2)
        Kminus = sqrt((Kx - Qx) ** 2 + (Ky - Qy) ** 2)
        K = sqrt(Kx ** 2 + Ky ** 2)
        bright_field = K < alpha / lam
        double_overlap1 = (Kplus < alpha / lam) * bright_field * (Kminus > alpha / lam)
        double_overlap2 = (Kplus > alpha / lam) * bright_field * (Kminus < alpha / lam)

        Γ_abs = abs(Γ)
        take = Γ_abs > eps and bright_field
        if take:
            val = G[iqy, iqx, iky, ikx] * Γ.conjugate()
            cuda.atomic.add(Ψ_Qp.real, (iqy, iqx), val.real)
            cuda.atomic.add(Ψ_Qp.imag, (iqy, iqx), val.imag)
        if double_overlap1:
            val = G[iqy, iqx, iky, ikx] * Γ.conjugate()
            cuda.atomic.add(Ψ_Qp_left_sb.real, (iqy, iqx), val.real)
            cuda.atomic.add(Ψ_Qp_left_sb.imag, (iqy, iqx), val.imag)
        if double_overlap2:
            val = G[iqy, iqx, iky, ikx] * Γ.conjugate()
            cuda.atomic.add(Ψ_Qp_right_sb.real, (iqy, iqx), val.real)
            cuda.atomic.add(Ψ_Qp_right_sb.imag, (iqy, iqx), val.imag)
        if iqx == 0 and iqy == 0:
            val = abs(G[iqy, iqx, iky, ikx]) + 1j * 0
            cuda.atomic.add(Ψ_Qp.real, (iqy, iqx), val.real)
            cuda.atomic.add(Ψ_Qp_left_sb.real, (iqy, iqx), val.real)
            cuda.atomic.add(Ψ_Qp_right_sb.real, (iqy, iqx), val.real)

In [ ]:
#export

import matplotlib
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
import numpy as np
from PIL import Image

def sector_mask(shape, centre, radius, angle_range=(0,360)):
    """
    Return a boolean mask for a circular sector. The start/stop angles in
    `angle_range` should be given in clockwise order.
    """

    x, y = np.ogrid[:shape[0], :shape[1]]
    cx, cy = centre
    tmin, tmax = np.deg2rad(angle_range)

    # ensure stop angle > start angle
    if tmax < tmin:
        tmax += 2 * np.pi

    # convert cartesian --> polar coordinates
    r2 = (x - cx) * (x - cx) + (y - cy) * (y - cy)
    theta = np.arctan2(x - cx, y - cy) - tmin

    # wrap angles between 0 and 2*pi
    theta %= (2 * np.pi)

    # circular mask
    circmask = r2 <= radius * radius

    # angular mask
    anglemask = theta <= (tmax - tmin)

    return circmask * anglemask

def HSV_to_RGB(cin):
    """\
    HSV to RGB transformation.
    """

    # HSV channels
    h, s, v = cin

    i = (6. * h).astype(int)
    f = (6. * h) - i
    p = v * (1. - s)
    q = v * (1. - s * f)
    t = v * (1. - s * (1. - f))
    i0 = (i % 6 == 0)
    i1 = (i == 1)
    i2 = (i == 2)
    i3 = (i == 3)
    i4 = (i == 4)
    i5 = (i == 5)

    imout = np.zeros(h.shape + (3,), dtype=h.dtype)
    imout[:, :, 0] = 255 * (i0 * v + i1 * q + i2 * p + i3 * p + i4 * t + i5 * v)
    imout[:, :, 1] = 255 * (i0 * t + i1 * v + i2 * v + i3 * q + i4 * p + i5 * p)
    imout[:, :, 2] = 255 * (i0 * p + i1 * p + i2 * t + i3 * v + i4 * v + i5 * q)

    return imout


def zplot(imgs, suptitle='Image', savePath=None, cmap=['hot', 'hsv'], title=['Abs', 'Phase'], show=True,
          figsize=(9, 5), scale=None):
    im1, im2 = imgs
    fig = plt.figure(figsize=figsize, dpi=300)
    fig.suptitle(suptitle, fontsize=15, y=0.8)
    gs1 = gridspec.GridSpec(1, 2)
    gs1.update(wspace=0, hspace=0)  # set the spacing between axes.
    ax1 = plt.subplot(gs1[0])
    ax2 = plt.subplot(gs1[1])
    div1 = make_axes_locatable(ax1)
    div2 = make_axes_locatable(ax2)

    imax1 = ax1.imshow(im1, interpolation='nearest', cmap=plt.cm.get_cmap(cmap[0]))
    imax2 = ax2.imshow(im2, interpolation='nearest', cmap=plt.cm.get_cmap(cmap[1]))

    cax1 = div1.append_axes("left", size="10%", pad=0.4)
    cax2 = div2.append_axes("right", size="10%", pad=0.4)

    cbar1 = plt.colorbar(imax1, cax=cax1)
    cbar2 = plt.colorbar(imax2, cax=cax2)

    cax1.yaxis.set_ticks_position('left')
    ax2.yaxis.set_ticks_position('right')

    ax1.set_title(title[0])
    ax2.set_title(title[1])

    if scale is not None:
        fontprops = fm.FontProperties(size=18)
        scalebar = AnchoredSizeBar(ax1.transData,
                                   scale[0], scale[1], 'lower right',
                                   pad=0.1,
                                   color='white',
                                   frameon=False,
                                   size_vertical=im1.shape[0] / 40,
                                   fontproperties=fontprops)

        ax1.add_artist(scalebar)

    ax1.grid(False)
    ax2.grid(False)

    if show:
        plt.show()
    if savePath is not None:
        # print 'saving'
        fig.savefig(savePath + '.png')


def plotAbsAngle(img, suptitle='Image', savePath=None, cmap=['gray', 'gray'], title=['Abs', 'Phase'], show=True,
                 figsize=(10, 10), scale=None):
    zplot([np.abs(img), np.angle(img)], suptitle, savePath, cmap, title, show, figsize, scale)


def plotcxmosaic(img, title='Image', savePath=None, cmap='hot', show=True, figsize=(10, 10), vmax=None):
    fig, ax = plt.subplots(figsize=figsize)
    mos = imsave(mosaic(img))
    cax = ax.imshow(mos, interpolation='nearest', cmap=plt.cm.get_cmap(cmap), vmax=vmax)
    ax.set_title(title)
    plt.grid(False)
    plt.show()
    if savePath is not None:
        # print 'saving'
        fig.savefig(savePath + '.png', dpi=600)


def plotcxmosaic(img, title='Image', savePath=None, cmap='hot', show=True, figsize=(10, 10), vmax=None):
    fig, ax = plt.subplots(figsize=figsize)
    mos = imsave(mosaic(img))
    cax = ax.imshow(mos, interpolation='nearest', cmap=plt.cm.get_cmap(cmap), vmax=vmax)
    ax.set_title(title)
    plt.grid(False)
    plt.show()
    if savePath is not None:
        # print 'saving'
        fig.savefig(savePath + '.png', dpi=600)


def plotmosaic(img, title='Image', savePath=None, cmap='hot', show=True, figsize=(10, 10), vmax=None):
    fig, ax = plt.subplots(figsize=figsize)
    mos = mosaic(img)
    cax = ax.imshow(mos, interpolation='nearest', cmap=plt.cm.get_cmap(cmap), vmax=vmax)
    cbar = fig.colorbar(cax)
    ax.set_title(title)
    plt.grid(False)
    plt.show()
    if savePath is not None:
        # print 'saving'
        fig.savefig(savePath + '.png', dpi=600)
    return fig


def plot(img, title='Image', savePath=None, cmap='inferno', show=True, vmax=None, figsize=(10, 10), scale=None):
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(img, interpolation='nearest', cmap=plt.cm.get_cmap(cmap), vmax=vmax)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im, cax=cax)
    ax.set_title(title)
    fontprops = fm.FontProperties(size=18)
    if scale is not None:
        scalebar = AnchoredSizeBar(ax.transData,
                                   scale[0], scale[1], 'lower right',
                                   pad=0.1,
                                   color='white',
                                   frameon=False,
                                   size_vertical=img.shape[0] / 40,
                                   fontproperties=fontprops)

        ax.add_artist(scalebar)
    ax.grid(False)
    if savePath is not None:
        fig.savefig(savePath + '.pdf', dpi=600)
        # fig.savefig(savePath + '.eps', dpi=600)
    if show:
        plt.show()
    return fig


def mosaic(data):
    n, w, h = data.shape
    diff = np.sqrt(n) - int(np.sqrt(n))
    s = np.sqrt(n)
    m = int(s)
    # print 'm', m
    if diff > 1e-6: m += 1
    mosaic = np.zeros((m * w, m * h)).astype(data.dtype)
    for i in range(m):
        for j in range(m):
            if (i * m + j) < n:
                mosaic[i * w:(i + 1) * w, j * h:(j + 1) * h] = data[i * m + j]
    return mosaic


def P1A_to_HSV(cin, vmin=None, vmax=None):
    """\
    Transform a complex array into an RGB image,
    mapping phase to hue, amplitude to value and
    keeping maximum saturation.
    """
    # HSV channels
    h = .5 * np.angle(cin) / np.pi + .5
    s = np.ones(cin.shape)

    v = abs(cin)
    if vmin is None: vmin = 0.
    if vmax is None: vmax = v.max()
    assert vmin < vmax
    v = (v.clip(vmin, vmax) - vmin) / (vmax - vmin)

    return HSV_to_RGB((h, s, v))


def imsave(a, filename=None, vmin=None, vmax=None, cmap=None):
    """
    imsave(a) converts array a into, and returns a PIL image
    imsave(a, filename) returns the image and also saves it to filename
    imsave(a, ..., vmin=vmin, vmax=vmax) clips the array to values between vmin and vmax.
    imsave(a, ..., cmap=cmap) uses a matplotlib colormap.
    """

    if a.dtype.kind == 'c':
        # Image is complex
        if cmap is not None:
            print('imsave: Ignoring provided cmap - input array is complex')
        i = P1A_to_HSV(a, vmin, vmax)
        im = Image.fromarray(np.uint8(i), mode='RGB')

    else:
        if vmin is None:
            vmin = a.min()
        if vmax is None:
            vmax = a.max()
        im = Image.fromarray((255 * (a.clip(vmin, vmax) - vmin) / (vmax - vmin)).astype('uint8'))
        if cmap is not None:
            r = im.point(lambda x: cmap(x / 255.0)[0] * 255)
            g = im.point(lambda x: cmap(x / 255.0)[1] * 255)
            b = im.point(lambda x: cmap(x / 255.0)[2] * 255)
            im = Image.merge("RGB", (r, g, b))
            # b = (255*(a.clip(vmin,vmax)-vmin)/(vmax-vmin)).astype('uint8')
            # im = Image.fromstring('L', a.shape[-1::-1], b.tostring())

    if filename is not None:
        im.save(filename)
    return im


def plot_complex_multi(x, title='_', figsize=(10, 10), savePath=None, scale=None, show=True):
    n, h, w = x.shape
    rows = int(np.floor(np.sqrt(n)))
    cols = n // rows + 1
    fontprops = fm.FontProperties(size=18)
    fig = plt.figure(dpi=300, constrained_layout=True)
    gs1 = gridspec.GridSpec(rows, cols)
    gs1.update(wspace=0.1, hspace=0.1)
    for r in range(rows):
        for c in range(cols):
            i = cols * r + c
            ax = plt.subplot(gs1[i])
            if i < n:
                imax1 = ax.imshow(imsave(x[i]), interpolation='nearest')
                if scale is not None and i == 0:
                    scalebar = AnchoredSizeBar(ax.transData,
                                               scale[0], scale[1], 'lower right',
                                               pad=0.1,
                                               color='white',
                                               frameon=False,
                                               size_vertical=x.shape[0] / 40,
                                               fontproperties=fontprops)
                    ax.add_artist(scalebar)
            else:
                ax.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_xticks([])
            ax.set_yticks([])

    plt.suptitle(title)
    plt.grid(False)
    fig.set_constrained_layout_pads(w_pad=0.1, h_pad=0.1)
    if savePath is not None:
        fig.savefig(savePath + '.png')
        fig.savefig(savePath + '.pdf')
    if show:
        plt.show()
    return fig


def plotcx(x, title='Image', figsize=(10, 10), savePath=None, scale=None):
    fig, (ax1) = plt.subplots(1, 1, figsize=figsize)
    imax1 = ax1.imshow(imsave(x), interpolation='nearest')
    ax1.set_title(title)
    plt.grid(False)
    fontprops = fm.FontProperties(size=18)
    if scale is not None:
        scalebar = AnchoredSizeBar(ax1.transData,
                                   scale[0], scale[1], 'lower right',
                                   pad=0.1,
                                   color='white',
                                   frameon=False,
                                   size_vertical=x.shape[0] / 40,
                                   fontproperties=fontprops)

        ax1.add_artist(scalebar)
    if savePath is not None:
        fig.savefig(savePath + '.png', dpi=300)
        fig.savefig(savePath + '.pdf', dpi=300)
    plt.show()

In [ ]:
#export
from numba import cuda
from math import *

def disk_overlap_function(Qx_all, Qy_all, Kx_all, Ky_all, aberrations, theta_rot, alpha, lam):
    n_batch = Qx_all.shape[0]
    xp = cp.get_array_module(Qx_all)
    Gamma = xp.zeros((n_batch,) + (Ky_all.shape[0], Kx_all.shape[0]), dtype=xp.complex64)
    gs = Gamma.shape
    threadsperblock = 2 ** 8
    blockspergrid = m.ceil(np.prod(gs) / threadsperblock)
    strides = cp.array((np.array(Gamma.strides) / (Gamma.nbytes / Gamma.size)).astype(np.int))
    disk_overlap_kernel[blockspergrid, threadsperblock](Gamma, strides, Qx_all, Qy_all, Kx_all, Ky_all, aberrations,
                                                        theta_rot, alpha, lam)
    cp.cuda.Device().synchronize()
    return Gamma


@cuda.jit
def disk_overlap_kernel(Γ, strides, Qx_all, Qy_all, Kx_all, Ky_all, aberrations, theta_rot, alpha, lam):
    def aperture2(qx, qy, lam, alpha_max):
        qx2 = qx ** 2
        qy2 = qy ** 2
        q = sqrt(qx2 + qy2)
        ktheta = m.asin(q * lam)
        return ktheta < alpha_max

    def chi3(qy, qx, lam, C):
        """
        Zernike polynomials in the cartesian coordinate system
        :param qx:
        :param qy:
        :param lam: wavelength in Angstrom
        :param C:   (12 ,)
        :return:
        """

        u = qx * lam
        v = qy * lam
        u2 = u ** 2
        u3 = u ** 3
        u4 = u ** 4
        # u5 = u ** 5

        v2 = v ** 2
        v3 = v ** 3
        v4 = v ** 4
        # v5 = v ** 5

        # aberr = Param()
        # aberr.C1 = C[0]
        # aberr.C12a = C[1]
        # aberr.C12b = C[2]
        # aberr.C21a = C[3]
        # aberr.C21b = C[4]
        # aberr.C23a = C[5]
        # aberr.C23b = C[6]
        # aberr.C3 = C[7]
        # aberr.C32a = C[8]
        # aberr.C32b = C[9]
        # aberr.C34a = C[10]
        # aberr.C34b = C[11]

        chi = 0

        # r-2 = x-2 +y-2.
        chi += 1 / 2 * C[0] * (u2 + v2)  # r^2
        # r-2 cos(2*phi) = x"2 -y-2.
        # r-2 sin(2*phi) = 2*x*y.
        chi += 1 / 2 * (C[1] * (u2 - v2) + 2 * C[2] * u * v)  # r^2 cos(2 phi) + r^2 sin(2 phi)
        # r-3 cos(3*phi) = x-3 -3*x*y'2. r"3 sin(3*phi) = 3*y*x-2 -y-3.
        chi += 1 / 3 * (C[5] * (u3 - 3 * u * v2) + C[6] * (3 * u2 * v - v3))  # r^3 cos(3phi) + r^3 sin(3 phi)
        # r-3 cos(phi) = x-3 +x*y-2.
        # r-3 sin(phi) = y*x-2 +y-3.
        chi += 1 / 3 * (C[3] * (u3 + u * v2) + C[4] * (v3 + u2 * v))  # r^3 cos(phi) + r^3 sin(phi)
        # r-4 = x-4 +2*x-2*y-2 +y-4.
        chi += 1 / 4 * C[7] * (u4 + v4 + 2 * u2 * v2)  # r^4
        # r-4 cos(4*phi) = x-4 -6*x-2*y-2 +y-4.
        chi += 1 / 4 * C[10] * (u4 - 6 * u2 * v2 + v4)  # r^4 cos(4 phi)
        # r-4 sin(4*phi) = 4*x-3*y -4*x*y-3.
        chi += 1 / 4 * C[11] * (4 * u3 * v - 4 * u * v3)  # r^4 sin(4 phi)
        # r-4 cos(2*phi) = x-4 -y-4.
        chi += 1 / 4 * C[8] * (u4 - v4)
        # r-4 sin(2*phi) = 2*x-3*y +2*x*y-3.
        chi += 1 / 4 * C[9] * (2 * u3 * v + 2 * u * v3)
        # r-5 cos(phi) = x-5 +2*x-3*y-2 +x*y-4.
        # r-5 sin(phi) = y*x"4 +2*x-2*y-3 +y-5.
        # r-5 cos(3*phi) = x-5 -2*x-3*y-2 -3*x*y-4.
        # r-5 sin(3*phi) = 3*y*x-4 +2*x-2*y-3 -y-5.
        # r-5 cos(5*phi) = x-5 -10*x-3*y-2 +5*x*y-4.
        # r-5 sin(5*phi) = 5*y*x-4 -10*x-2*y-3 +y-5.

        chi *= 2 * np.pi / lam

        return chi

    gs = Γ.shape
    N = gs[0] * gs[1] * gs[2]
    n = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    j = n // strides[0]
    iky = (n - j * strides[0]) // strides[1]
    ikx = (n - (j * strides[0] + iky * strides[1])) // strides[2]

    if n < N:
        Qx = Qx_all[j]
        Qy = Qy_all[j]
        Kx = Kx_all[ikx]
        Ky = Ky_all[iky]

        Qx_rot = Qx * cos(theta_rot) - Qy * sin(theta_rot)
        Qy_rot = Qx * sin(theta_rot) + Qy * cos(theta_rot)

        Qx = Qx_rot
        Qy = Qy_rot

        chi = chi3(Ky, Kx, lam, aberrations)
        A = aperture2(Ky, Kx, lam, alpha) * cm.exp(-1j * chi)
        chi = chi3(Ky + Qy, Kx + Qx, lam, aberrations)
        Ap = aperture2(Ky + Qy, Kx + Qx, lam, alpha) * cm.exp(-1j * chi)
        chi = chi3(Ky - Qy, Kx - Qx, lam, aberrations)
        Am = aperture2(Ky - Qy, Kx - Qx, lam, alpha) * cm.exp(-1j * chi)

        Γ[j, iky, ikx] = A.conjugate() * Am - A * Ap.conjugate()

In [ ]:
#export
from math import sqrt
def wavelength(E_eV):
    emass = 510.99906;  # electron rest mass in keV
    hc = 12.3984244;  # h*c
    lam = hc / m.sqrt(E_eV * 1e-3 * (2 * emass + E_eV * 1e-3))  # in Angstrom
    return lam  


def DOF(alpha, E_eV):
    E0 = E_eV

    # Calculate wavelength and electron interaction parameter
    m = 9.109383 * 10 ** -31
    e = 1.602177 * 10 ** -19
    c = 299792458
    h = 6.62607 * 10 ** -34

    lam = h / sqrt(2 * m * e * E0) / sqrt(1 + e * E0 / 2 / m / c ** 2) * 10 ** 10
    DOF = 2 * lam / alpha ** 2
    return DOF

def get_qx_qy_1D(M, dx, dtype, fft_shifted=False):
    qxa = cp.fft.fftfreq(M[0], dx[0]).astype(dtype)
    qya = cp.fft.fftfreq(M[1], dx[1]).astype(dtype)
    if fft_shifted:
        qxa = cp.fft.fftshift(qxa)
        qya = cp.fft.fftshift(qya)
    return qxa, qya


def get_qx_qy_2D(M, dx, dtype, fft_shifted=False):
    qxa = cp.fft.fftfreq(M[0], dx[0]).astype(dtype)
    qya = cp.fft.fftfreq(M[1], dx[1]).astype(dtype)
    [qxn, qyn] = cp.meshgrid(qxa, qya)
    if fft_shifted:
        qxn = cp.fft.fftshift(qxn)
        qyn = cp.fft.fftshift(qyn)
    return qxn, qyn

def get_qx_qy_1D_th(M, dx, dtype, fft_shifted=False):
    qxa = np.fft.fftfreq(M[0], dx[0]).astype(dtype)
    qya = np.fft.fftfreq(M[1], dx[1]).astype(dtype)
    if fft_shifted:
        qxa = np.fft.fftshift(qxa)
        qya = np.fft.fftshift(qya)
    return th.as_tensor(np.stack([qxa, qya]))

def get_qx_qy_2D_th(M, dx, dtype, fft_shifted=False):
    qxa = np.fft.fftfreq(M[0], dx[0]).astype(dtype)
    qya = np.fft.fftfreq(M[1], dx[1]).astype(dtype)
    [qxn, qyn] = np.meshgrid(qxa, qya)
    if fft_shifted:
        qxn = np.fft.fftshift(qxn)
        qyn = np.fft.fftshift(qyn)
    return th.as_tensor(np.stack([qxn, qyn]))

In [ ]:
#export 
from numba import cuda
@cuda.jit
def center_of_mass_kernel(comx, comy, indices, counts, frame_dimensions, no_count_indicator, qx, qy):
    ny, nx = cuda.grid(2)
    NY, NX, _ = indices.shape
    MY, MX = frame_dimensions
    if ny < NY and nx < NX:
        for i in range(indices[ny, nx].shape[0]):
            idx1d = indices[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            if idx1d != no_count_indicator:
                cuda.atomic.add(comy, (ny, nx), counts[ny, nx, i] * qy[my, mx])
                cuda.atomic.add(comx, (ny, nx), counts[ny, nx, i] * qx[my, mx])

In [ ]:
#export 
from numba import cuda
@cuda.jit
def sparse_to_dense_datacube_kernel_crop(dc, indices, counts, frame_dimensions, bin, start, end, no_count_indicator):
    ny, nx = cuda.grid(2)
    NY, NX, MYBIN, MXBIN = dc.shape
    MY, MX = frame_dimensions
    if ny < NY and nx < NX:
        for i in range(indices[ny, nx].shape[0]):
            idx1d = indices[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            if my >= start[0] and mx >= start[1] and my < end[0] and mx < end[1]:
                mybin = (my - start[0]) // bin
                mxbin = (mx - start[1]) // bin
                if idx1d != no_count_indicator:
                    cuda.atomic.add(dc, (ny, nx, mybin, mxbin), counts[ny, nx, i])

In [ ]:
#export
import numpy as np
import cupy as cp
def sparse_to_dense_datacube_crop(indices, counts, scan_dimensions, frame_dimensions, center, radius, bin=1):
    radius = int(np.ceil(radius / bin) * bin)
    start = center - radius
    end = center + radius
    frame_size = 2 * radius // bin

    xp = cp.get_array_module(indices)
    dc = cp.zeros((*scan_dimensions, frame_size, frame_size), dtype=indices.dtype)

    threadsperblock = (16, 16)
    blockspergrid = tuple(np.ceil(np.array(indices.shape[:2]) / threadsperblock).astype(np.int))

    no_count_indicator = np.iinfo(indices.dtype).max

    sparse_to_dense_datacube_kernel_crop[blockspergrid, threadsperblock](dc, indices, counts, cp.array(frame_dimensions), bin,
                                                                         start, end, no_count_indicator)
    return dc

    

In [ ]:
# import smatrix as sm
# center, radius = sm.util.determine_center_and_radius(data, manual=False) 


# import cupy as cp
# import numpy as np
# from skimage.filters import gaussian
# from matplotlib.patches import Circle
# 
# 
# sh = np.concatenate([d.scan_dimensions,d.frame_dimensions])
# size = 25
# c = np.zeros((2,))
# c[:] = (sh[-1] // 2, sh[-2] // 2)
# radius = np.ones((1,)) * sh[-1] // 2  
# # frames, scan_dimensions, frame_dimensions, center, radius, bin=1
# dc_subset = sparse_to_dense_datacube_crop(cp.array(d.frames[:size, :size].astype(np.uint32)), (size,size), d.frame_dimensions, c, radius, bin=1)
# dcs = cp.sum(dc_subset, (0, 1))
# m1 = dcs.get()
# m = (gaussian(m1.astype(np.float32),2) > m1.max() * 2e-1).astype(np.float)
# r, y0, x0 = get_probe_size(m)
# 
# fig, ax = plt.subplots(figsize=(10,10))
# ax.matshow(m1)
# ax.scatter(x0,y0, s=10, c='r')
# c = Circle((x0,y0),r, fill=None)
# ax.add_patch(c)

In [ ]:
#export
from numba import cuda
@cuda.jit
def sparse_to_dense_datacube_crop_gain_mask_kernel(dc, frames, counts,
                                                   frame_dimensions,
                                                   center_frame, center_data,
                                                   radius_data_int, binning,
                                                   fftshift):
    ny, nx = cuda.grid(2)
    NY, NX, MYBIN, MXBIN = dc.shape
    MY, MX = frame_dimensions
    if ny < NY and nx < NX:
        for i in range(frames[ny, nx].shape[0]):
            idx1d = frames[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            my_center = my - center_data[0]
            mx_center = mx - center_data[1]
            dist_center = m.sqrt(my_center ** 2 + mx_center ** 2)
            if dist_center < radius_data_int:
                mybin = int(center_frame[0] + my_center // binning)
                mxbin = int(center_frame[1] + mx_center // binning)
                if fftshift:
                    mybin = (mybin - center_frame[0]) % (center_frame[0] * 2)
                    mxbin = (mxbin - center_frame[1]) % (center_frame[1] * 2)
                if (mxbin >= 0 and mybin >= 0):
                    cuda.atomic.add(dc, (ny, nx, mybin, mxbin), counts[ny, nx, i])

In [ ]:
#export
import cupy as cp
import numpy as np
import torch as th
import numba.cuda as cuda
def sparse_to_dense_datacube_crop_gain_mask(indices, counts, scan_dimensions, frame_dimensions, center_data, radius_data,
                                            radius_max, binning=1, fftshift=False):
    radius_data_int = int(np.ceil(radius_data / binning) * binning)
    radius_max_int = int(np.ceil(radius_max / binning) * binning)
    frame_size = 2 * radius_max_int // binning
    
    print(f'radius_data_int : {radius_data_int} ')
    print(f'radius_max_int  : {radius_max_int} ')
    print(f'Dense frame size: {frame_size}x {frame_size}')
    
    stream = th.cuda.current_stream().cuda_stream
    
    dc0 = np.zeros((scan_dimensions[0],scan_dimensions[1], frame_size, frame_size), dtype=np.uint8) 
    dc = th.zeros((scan_dimensions[0]//2,scan_dimensions[1], frame_size, frame_size), dtype=th.float32) 
    
    center_frame = th.tensor([frame_size // 2, frame_size // 2])
    fd = th.as_tensor(frame_dimensions) 
    center = th.as_tensor(center_data) 
    inds = th.as_tensor(indices[:scan_dimensions[0]//2,...]) 
    cts = th.as_tensor(counts[:scan_dimensions[0]//2,...].astype(np.float32), dtype=th.float32) 

    threadsperblock = (16, 16)
    blockspergrid = tuple(np.ceil(np.array(indices.shape[:2]) / threadsperblock).astype(np.int))
    print('sparse_to_dense_datacube_crop_gain_mask dtypes:',dc.dtype, inds.dtype, cts.dtype, frame_dimensions.dtype)
    
    sparse_to_dense_datacube_crop_gain_mask_kernel[blockspergrid, threadsperblock, stream](dc, inds, cts, fd,
                                                                                   center_frame, center,
                                                                                   radius_data_int, binning,
                                                                                   fftshift)
    
    dc0[:scan_dimensions[0]//2,...] = dc.cpu().type(th.uint8).numpy()
    
    dc[:] = 0 
    inds = th.as_tensor(indices[scan_dimensions[0]//2:,...]) 
    cts = th.as_tensor(counts[scan_dimensions[0]//2:,...].astype(np.float32), dtype=th.float32) 
    
    sparse_to_dense_datacube_crop_gain_mask_kernel[blockspergrid, threadsperblock, stream](dc, inds, cts, fd,
                                                                                   center_frame, center,
                                                                                   radius_data_int, binning,
                                                                                   fftshift)
    dc0[scan_dimensions[0]//2:,...] = dc.cpu().type(th.uint8).numpy()
    cuda.select_device(0)
    return dc0

In [ ]:
#export
import cupy as cp
import numpy as np
import torch as th
import numba.cuda as cuda
def sparse_to_dense_datacube(indices, counts, scan_dimensions, frame_dimensions, center_data, radius_data,
                                            radius_max, binning=1, fftshift=False):
    radius_data_int = int(np.ceil(radius_data / binning) * binning)
    radius_max_int = int(np.ceil(radius_max / binning) * binning)
    frame_size = 2 * radius_max_int // binning
    
    print(f'radius_data_int : {radius_data_int} ')
    print(f'radius_max_int  : {radius_max_int} ')
    print(f'dense frame size: {frame_size}x {frame_size}')
    
    dc = cp.zeros((scan_dimensions[0],scan_dimensions[1], frame_size, frame_size), dtype=cp.float32) 
    
    center_frame = cp.array(([frame_size // 2, frame_size // 2]))
    fd = cp.array(frame_dimensions) 
    center = cp.array(center_data) 

    threadsperblock = (16, 16)
    blockspergrid = tuple(np.ceil(np.array(indices.shape[:2]) / threadsperblock).astype(np.int))
    
    sparse_to_dense_datacube_crop_gain_mask_kernel[blockspergrid, threadsperblock](dc, indices, counts, fd,
        center_frame, center, radius_data_int, binning, fftshift)    
    return dc

In [ ]:
#export
from numba import cuda
@cuda.jit
def fftshift_kernel(indices, center_frame, scan_dimensions, no_count_indicator):
    ny, nx = cuda.grid(2)
    NY, NX = scan_dimensions
    MY = center_frame[0] * 2
    MX = center_frame[1] * 2
    if ny < NY and nx < NX:
        for i in range(indices[ny, nx].shape[0]):
            idx1d = indices[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            mysh = (my - center_frame[0]) % (center_frame[0] * 2)
            mxsh = (mx - center_frame[1]) % (center_frame[1] * 2)
            if idx1d != no_count_indicator:
                indices[ny, nx, i] = mysh * MX + mxsh

In [ ]:
#export
from numba import cuda
@cuda.jit
def fftshift_pad_kernel(indices, center_frame, scan_dimensions, new_frame_dimensions, no_count_indicator_old, no_count_indicator_new):
    ny, nx = cuda.grid(2)
    NY, NX = scan_dimensions
    MX = center_frame[1] * 2
    if ny < NY and nx < NX:
        for i in range(indices[ny, nx].shape[0]):
            idx1d = indices[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            mysh = (my - center_frame[0]) % (new_frame_dimensions[0])
            mxsh = (mx - center_frame[1]) % (new_frame_dimensions[1])
            if idx1d != no_count_indicator_old:
                indices[ny, nx, i] = mysh * new_frame_dimensions[1] + mxsh
            else:
                indices[ny, nx, i] = no_count_indicator_new

In [ ]:
#export
from numba import cuda
@cuda.jit
def virtual_annular_image_kernel(img, indices, counts, radius_inner, radius_outer, center_frame, frame_dimensions, no_count_indicator):
    ny, nx = cuda.grid(2)
    NY, NX, _ = indices.shape
    MY, MX = frame_dimensions
    if ny < NY and nx < NX:
        for i in range(indices[ny, nx].shape[0]):
            idx1d = indices[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            my_center = my - center_frame[0]
            mx_center = mx - center_frame[1]
            radius = m.sqrt(my_center ** 2 + mx_center ** 2)
            if radius < radius_outer and radius >= radius_inner and idx1d != no_count_indicator:
                cuda.atomic.add(img, (ny,nx), counts[ny, nx, i])

In [ ]:
#export 
from numba import cuda
import math as m

@cuda.jit
def crop_symmetric_around_center_kernel(new_frames, old_frames, center_frame, old_frame_dimensions, center_data, radius_data_int):
    ny, nx = cuda.grid(2)
    NY, NX, _ = old_frames.shape
    MY, MX = old_frame_dimensions
    MXnew = center_frame[1] * 2
    if ny < NY and nx < NX:
        k = 0
        for i in range(old_frames[ny, nx].shape[0]):
            idx1d = old_frames[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            my_center = my - center_data[0]
            mx_center = mx - center_data[1]
            dist_center = m.sqrt(my_center ** 2 + mx_center ** 2)
            if dist_center < radius_data_int:
                mybin = int(center_frame[0] + my_center)
                mxbin = int(center_frame[1] + mx_center)
                new_frames[ny, nx, k] = mybin * MXnew + mxbin
                k += 1

In [ ]:
#export
import cupy as cp
import numpy as np
def crop_symmetric_around_center(old_frames, old_frame_dimensions, center_data, max_radius):
    max_radius_int = int(max_radius)
    frame_size = 2 * max_radius_int
    center_frame = cp.array([frame_size // 2, frame_size // 2])
    new_frame_dimensions = np.array([frame_size,frame_size])

    threadsperblock = (16, 16)
    blockspergrid = tuple(np.ceil(np.array(old_frames.shape[:2]) / threadsperblock).astype(np.int))

    new_frames = cp.zeros_like(old_frames)
    new_frames[:] = cp.iinfo(new_frames.dtype).max

    crop_symmetric_around_center_kernel[blockspergrid, threadsperblock](new_frames, old_frames, center_frame,
                                                                        cp.array(old_frame_dimensions),
                                                                        cp.array(center_data), max_radius_int)

    max_counts = cp.max(cp.sum(new_frames > 0, 2).ravel())
    res = cp.ascontiguousarray(new_frames[:,:,:max_counts])
    return res, new_frame_dimensions

In [ ]:
#export 
from numba import cuda
import math as m

@cuda.jit
def rotate_kernel(frames, center_frame, old_frame_dimensions, center_data, no_count_indicator, angle_rad):
    ny, nx = cuda.grid(2)
    NY, NX, _ = frames.shape
    MY, MX = old_frame_dimensions
    MXnew = center_frame[1] * 2
    if ny < NY and nx < NX:
        for i in range(frames[ny, nx].shape[0]):
            idx1d = frames[ny, nx, i]
            if idx1d != no_count_indicator:
                my = idx1d // MX
                mx = idx1d - my * MX
                my_center = my - center_data[0]
                mx_center = mx - center_data[1]
                #rotate
                mx_center_rot = round(mx_center * m.cos(angle_rad) - my_center * m.sin(angle_rad))
                my_center_rot = round(mx_center * m.sin(angle_rad) + my_center * m.cos(angle_rad))
                mybin = int(center_frame[0] + my_center_rot)
                mxbin = int(center_frame[1] + mx_center_rot)
                frames[ny, nx, i] = mybin * MXnew + mxbin

In [ ]:
#export
import cupy as cp
import numpy as np
def rotate(frames, old_frame_dimensions, center, angle_rad):
    threadsperblock = (16, 16)
    blockspergrid = tuple(np.ceil(np.array(frames.shape[:2]) / threadsperblock).astype(np.int))
    no_count_indicator = np.iinfo(frames.dtype).max
    new_frames= cp.array(frames)
    rotate_kernel[blockspergrid, threadsperblock](new_frames, center, cp.array(old_frame_dimensions), cp.array(center),
                                                  no_count_indicator, angle_rad)
    
    return new_frames.get() 

In [ ]:
#export 
from numba import cuda
import math as m

@cuda.jit
def sum_kernel(indices, counts, frame_dimensions, sum, no_count_indicator):
    ny, nx = cuda.grid(2)
    NY, NX, _ = indices.shape
    MY, MX = frame_dimensions
    if ny < NY and nx < NX:
        for i in range(indices[ny, nx].shape[0]):
            idx1d = indices[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            if idx1d != no_count_indicator:
                cuda.atomic.add(sum, (my, mx), counts[ny, nx, i])

In [ ]:
#export
import cupy as cp
import numpy as np
def sum_frames(frames, counts, frame_dimensions):
    threadsperblock = (16, 16)
    blockspergrid = tuple(np.ceil(np.array(frames.shape[:2]) / threadsperblock).astype(np.int))
    
    sum = cp.zeros(frame_dimensions)
    no_count_indicator = np.iinfo(frames.dtype).max
    frames1 = cp.array(frames)
    counts1 = cp.array(counts)
    sum_kernel[blockspergrid, threadsperblock](frames1, counts1, cp.array(frame_dimensions), sum, no_count_indicator)
    return sum.get() 

In [ ]:
#export 
from numba import cuda
import math as m

@cuda.jit
def rebin_kernel(indices, counts, new_frame_center, old_indices, old_counts, old_frame_center, no_count_indicator,
                 bin_factor):
    ny, nx = cuda.grid(2)
    NY, NX, _ = indices.shape
    MY = old_frame_center[0] * 2
    MX = old_frame_center[1] * 2
    MXnew = new_frame_center[1] * 2
    if ny < NY and nx < NX:
        k = 0
        for i in range(old_indices[ny, nx].shape[0]):
            idx1d = old_indices[ny, nx, i]
            my = idx1d // MX
            mx = idx1d - my * MX
            my_center = my - old_frame_center[0]
            mx_center = mx - old_frame_center[1]
            if idx1d != no_count_indicator:
                mybin = int(new_frame_center[0] + my_center // bin_factor)
                mxbin = int(new_frame_center[1] + mx_center // bin_factor)
                indices[ny, nx, k] = mybin * MXnew + mxbin
                counts[ny, nx, k] = old_counts[ny, nx, i]
                k += 1
                

In [ ]:
#export
import numpy as np
def array_split_divpoints_ntotal(Ntotal, indices_or_sections):
    """
    Split an array into multiple sub-arrays.
    Please refer to the ``split`` documentation.  The only difference
    between these functions is that ``array_split`` allows
    `indices_or_sections` to be an integer that does *not* equally
    divide the axis. For an array of length l that should be split
    into n sections, it returns l % n sub-arrays of size l//n + 1
    and the rest of size l//n.
    See Also
    --------
    split : Split array into multiple sub-arrays of equal size.
    Examples
    --------
    >>> x = np.arange(8.0)
    >>> np.array_split(x, 3)
        [array([0.,  1.,  2.]), array([3.,  4.,  5.]), array([6.,  7.])]
    >>> x = np.arange(7.0)
    >>> np.array_split(x, 3)
        [array([0.,  1.,  2.]), array([3.,  4.]), array([5.,  6.])]
    """
    try:
        # handle array case.
        Nsections = len(indices_or_sections) + 1
        div_points = [0] + list(indices_or_sections) + [Ntotal]
    except TypeError:
        # indices_or_sections is a scalar, not an array.
        Nsections = int(indices_or_sections)
        if Nsections <= 0:
            raise ValueError('number sections must be larger than 0.')
        Neach_section, extras = divmod(Ntotal, Nsections)
        section_sizes = ([0] +
                         extras * [Neach_section + 1] +
                         (Nsections - extras) * [Neach_section])
        div_points = np.array(section_sizes, dtype=np.intp).cumsum()

    return div_points

In [ ]:
#export
from numba import cuda
@cuda.jit
def dense_to_sparse_kernel(dense, indices, counts, frame_dimensions):
    ny, nx = cuda.grid(2)
    NY, NX, MYBIN, MXBIN = dense.shape
    MY, MX = frame_dimensions
    if ny < NY and nx < NX:
        k = 0
        for mx in range(MX):
            for my in range(MY):
                idx1d = my * MX + mx
                if dense[ny,nx,my,mx] > 0:
                    indices[ny,nx,k] = idx1d
                    counts[ny,nx,k] = dense[ny,nx,my,mx]
                    k += 1                    

def advanced_raster_scan(ny=10, nx=10, fast_axis=1, mirror=[1, 1], theta=0, dy=1, dx=1):
    """
    Generates as raster scan.
    
    Parameters
    ----------
    ny, nx : int
        Number of steps in *y* (vertical) and *x* (horizontal) direction
        *x* is the fast axis
        
    dy, dx : float
        Step size (grid spacinf) in *y* and *x*  
        
    Returns
    -------
    pos : ndarray
        A (N,2)-array of positions.
        
    Examples
    --------
    """
    iix, iiy = np.indices((nx, ny))
    if fast_axis != 1:
        tmp = iix
        iix = iiy
        iiy = tmp

    # print iix.shape, iiy.shape
    positions = np.array([(dx * i, dy * j) for i, j in zip(iix.ravel(), iiy.ravel())]).astype(np.float32)

    mins = np.array([positions[:, 0].min(), positions[:, 1].min()])
    maxs = np.array([positions[:, 0].max(), positions[:, 1].max()])

    center = mins + (maxs - mins) / 2.0
    positions -= center

    positions[:, 0] *= mirror[0]
    positions[:, 1] *= mirror[1]

    theta_rad = theta / 180.0 * np.pi
    R = np.array([[np.cos(theta_rad), -np.sin(theta_rad)],
                  [np.sin(theta_rad), np.cos(theta_rad)]])
    # rotate counterclockwise by theta
    positions = positions.dot(R)
    mins = np.array([positions[:, 0].min(), positions[:, 1].min()])
    positions -= mins
    return positions.astype(np.float32)